In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (1000, 1000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 2000


In [5]:
from itertools import product
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance
import gensim

Path to word2vec embeddings

In [6]:
word2vec_path = get_relative_path(prefix_path, DEAULT_WORDVECTORS)
os.path.exists(word2vec_path)

True

In [7]:
wordvectors = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

Load counts documents by word

In [8]:
docs_samples_word_doc_count_path = get_relative_path(data_path, DOCS_SAMPLES_WORD_DOC_COUNT)
samples_word_in_docs_count = load_pickle(docs_samples_word_doc_count_path)

Measure $idf(t, D) = log\frac{N}{\vert \left\{ d\in D:t\in d \right\} \vert}$

In [9]:
n_docs = len(docs_sample_a) + len(docs_sample_b)
t_idf = {t: np.log(n_docs/v) for t, v in samples_word_in_docs_count.items()}

In [10]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='kps-normalized'):
    return list(t for k in samples_content[index][field] for t in k.decode('utf-8').replace("-", " ").split())

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    # Term frequencies by document
    tf_a = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    tf_b = tf_doc(get_content_for(docs_sample_b[j], samples_content))
    # Term list
    t_list = list(set(tf_a) | set(tf_b))
    # TF-IDF
    tf_idf_a = np.zeros([len(t_list),])
    tf_idf_b = np.zeros([len(t_list),])
    # wordvector length 300
    centroid_a = mean = np.zeros(300, dtype=np.float64)
    centroid_b = mean = np.zeros(300, dtype=np.float64)
    for pos, t in enumerate(t_list):
        if t and t in wordvectors.vocab and t in t_idf:
            wv = wordvectors.get_vector(t)
            if t in tf_a:
                tf_idf_a[pos] = tf_a[t] * t_idf[t]
                centroid_a += wv * tf_idf_a[pos]
            if t in tf_b:
                tf_idf_b[pos] = tf_b[t] * t_idf[t]
                centroid_b += wv * tf_idf_b[pos]
            
    sum_tf_idf_a = sum(tf_idf_a)
    sum_tf_idf_b = sum(tf_idf_b)
    # Centroid
    centroid_a /= sum_tf_idf_a
    centroid_b /= sum_tf_idf_b
    
    # Measure
    measure_sim = 1.0 - cosine_distance(centroid_a, centroid_b)
    m_measure_sim[i, j] = measure_sim

print(m_measure_sim)

[[0.43115028 0.54634225 0.51371095 ... 0.51008934 0.2913128  0.38669381]
 [0.53935693 0.74775339 0.4639082  ... 0.62689215 0.42685427 0.46811497]
 [0.47276884 0.58106644 0.52075612 ... 0.60518409 0.4051908  0.60488185]
 ...
 [0.3643233  0.60972315 0.43664798 ... 0.57395037 0.35191613 0.50837997]
 [0.44619979 0.52402713 0.26508918 ... 0.43225973 0.65464446 0.47329157]
 [0.4767633  0.64211708 0.51579008 ... 0.67357213 0.44694537 0.55012258]]


In [11]:
docs_samples_measure_sim_path = get_relative_path(data_path, DOCS_SAMPLES_WORD2VEC_TFIDF_KPS)
save_pickle(m_measure_sim, docs_samples_measure_sim_path)

In [12]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_measure_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_measure_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_measure_sim)

[[0.42301795 0.57503812 0.41932352 ... 0.54249371 0.39724739 0.44289797]
 [0.52300512 0.71095772 0.51843745 ... 0.67072092 0.49114326 0.54758411]
 [0.49526525 0.67324896 0.49093984 ... 0.63514629 0.46509332 0.51854058]
 ...
 [0.44335411 0.60268249 0.43948207 ... 0.56857354 0.41634465 0.46418984]
 [0.3652919  0.49656703 0.36210162 ... 0.46846371 0.34303805 0.38245905]
 [0.52350031 0.71163086 0.51892831 ... 0.67135596 0.49160828 0.54810257]]


In [13]:
docs_samples_measure_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_WORD2VEC_TFIDF_UDV_KPS)
save_pickle(m_udv_measure_sim, docs_samples_measure_sim_udv_path)